# 4th_homework

### Import Section

In [1]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender

### Settings Section

In [2]:
# random_state_global = 0

### Path Section

In [3]:
PATH_DATA_RETAIL = r'../2th lesson/retail_train.csv'
PATH_DATA_PRODUCT = r'../2th lesson/product.csv'

## Tasks

1. Изучите структуру модуля src.
2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py.
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.recommenders.py.
4. Проверьте, что все модули корректно импортируются.
5. Если вы еще не прочитали статью о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте:
https://habr.com/ru/company/hh/blog/347276/

## 1. Изучите структуру модуля src

Для изучения процесса создания собственных пакетов был изучен официальный туториал:
https://packaging.python.org/en/latest/tutorials/packaging-projects/

На простом примере был создан собственный пакет "example-package-YOUR-USERNAME-HERE-0.0.1.tar.gz".

In [4]:
# Установка пакета-примера.
!pip install packaging_tutorial\dist\example-package-YOUR-USERNAME-HERE-0.0.1.tar.gz

Processing c:\users\zsv\pycharmprojects\geekbrains\recommender systems\4th lesson\packaging_tutorial\dist\example-package-your-username-here-0.0.1.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for example-package-YOUR-USERNAME-HERE: filename=example_package_YOUR_USERNAME_HERE-0.0.1-py3-none-any.whl size=2833 sha256=4e3ec59f02015be6668302d5ce4b91801d8f5552e7795981a7d3d37a696cfde8
  Stored in directory: c:\users\zsv\appdata\local\pip\cache\wheels\c8\ae\be\34e0faf9e292dccbe04c696986299791a50d9c91b710b6e303
Successfully built example-package-YOUR-USERNAME-HERE
  Attempting uninstall: 

In [5]:
# Проверка.
from example_package import example

example.add_one(2)

3

## 2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py.

Сперва реализуем функции в основном коде и проверим их работоспособность.

In [6]:
df_products = pd.read_csv(PATH_DATA_PRODUCT)

df_products.columns = [col.lower() for col in df_products.columns]
df_products = df_products.rename(columns={'product_id': 'item_id'})

In [7]:
df_products.head()

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ


In [8]:
df_retail = pd.read_csv(PATH_DATA_RETAIL)

In [9]:
df_retail.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [10]:
# Три последние недели будут использованы для валидации обученной модели.
test_size_weeks = 3

# Разбиение данных на обучающую и тестовую выборки.
df_train = df_retail[df_retail['week_no'] < df_retail['week_no'].max() - test_size_weeks].copy()
df_test = df_retail[df_retail['week_no'] >= df_retail['week_no'].max() - test_size_weeks].copy()

# Проверка вывод размерностей выборок.
df_train.shape, df_test.shape

((2278490, 12), (118314, 12))

In [11]:
# Формирование проверочной таблицы на валидационной выборке.
result_test = df_test.copy().groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Prefilter Items

In [12]:
# Функция для фильтрации продуктов.
def prefilter_items(retail, products=None, departments=[], items_too_cheap=[], items_too_expensive=[]):
    
    # Уберем самые популярные товары (их и так купят).
    popularity = retail.groupby('item_id')['user_id'].nunique().reset_index()
    popularity = popularity.rename(columns={'user_id': 'share_unique_users'})
    popularity['share_unique_users'] = popularity['share_unique_users'] / retail['user_id'].nunique()
    top_popular = popularity.query('share_unique_users > 0.5')['item_id']
    retail = retail[~retail['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят).
    top_not_popular = popularity.query('share_unique_users < 0.01')['item_id']
    retail = retail[~retail['item_id'].isin(top_not_popular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев.
    weeks_actual = retail['week_no'].unique()[-12:]
    items_actual = retail.loc[retail['week_no'].isin(weeks_actual), 'item_id'].unique()
    retail = retail[retail['item_id'].isin(items_actual)]
    
    # Уберем не интересные для рекоммендаций категории (department).
    if products is not None and departments != []:
        items_dep = products.loc[products['department'].isin(departments), 'item_id']
        retail = retail[retail['item_id'].isin(items_dep)]
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    retail = retail[~retail['item_id'].isin(items_too_cheap)]
    
    # Уберем слишком дорогие товары.
    retail = retail[~retail['item_id'].isin(items_too_expensive)]
    
    return retail

In [13]:
%%time


df_train_pf = prefilter_items(df_train)

print(f'Размерность таблицы до фильтрации:\t{df_train.shape}\n' \
      f'Размерность таблицы после фильтрации:\t{df_train_pf.shape}\n' \
      f'\n' \
      f'Количество продуктов до фильтрации:\t{df_train["item_id"].nunique()}\n' \
      f'Количество продуктов после фильтрации:\t{df_train_pf["item_id"].nunique()}\n'
     )

Размерность таблицы до фильтрации:	(2278490, 12)
Размерность таблицы после фильтрации:	(1690216, 12)

Количество продуктов до фильтрации:	86865
Количество продуктов после фильтрации:	10935

CPU times: total: 2.12 s
Wall time: 2.16 s


In [14]:
df_products['department'].value_counts().head()

GROCERY      39021
DRUG GM      31529
PRODUCE       3118
COSMETICS     3011
NUTRITION     2914
Name: department, dtype: int64

In [15]:
%%time


df_train_pf = prefilter_items(df_train, df_products, ['GROCERY', 'DRUG GM'])

print(f'Размерность таблицы до фильтрации:\t{df_train.shape}\n' \
      f'Размерность таблицы после фильтрации:\t{df_train_pf.shape}\n' \
      f'\n' \
      f'Количество продуктов до фильтрации:\t{df_train["item_id"].nunique()}\n' \
      f'Количество продуктов после фильтрации:\t{df_train_pf["item_id"].nunique()}\n'
     )

Размерность таблицы до фильтрации:	(2278490, 12)
Размерность таблицы после фильтрации:	(1261301, 12)

Количество продуктов до фильтрации:	86865
Количество продуктов после фильтрации:	9008

CPU times: total: 2.16 s
Wall time: 2.18 s


Функция фильтрации отсеивает:
- наиболее и наименее популярные товары;
- товары без продаж в течение 12 месяцев;
- при указании параметра, товары из других категорий;
- при указании параметров, слишком дешёвые и дорогие товары.

### Postfilter Items

In [16]:
def postfilter_items(user_id, recommednations):
    pass

## 3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.recommenders.py.

#### Precision@K

In [17]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

#### Get Recommendations

In [18]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

#### Лучшая модель из предыдущего урока

In [19]:
# Формирование таблицы, где для каждого товара указано количество продаж.
popularity = df_train.copy().groupby('item_id')['quantity'].sum().reset_index()
popularity = popularity.rename(columns={'quantity': 'n_sold'})

# Составление списка 5000 наиболее продаваемых товаров.
top_5000 = popularity.copy().sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Товары вне топ 5000 определены в отдельную категорию.
df_train.loc[~df_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [20]:
# Подготовка User-Item матрицы.
user_item_matrix = pd.pivot_table(df_train,
                                  index='user_id',
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='sum',
                                  fill_value=0
                                 )

# Необходимый тип матрицы для implicit.
user_item_matrix = user_item_matrix.astype(float) 

# Перевод в формат saprse matrix.
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Словари перевода user_id и item_id к порядковым id и наоброт
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [22]:
%%time


# Инициализация модели.
model_als = AlternatingLeastSquares(factors=10,
                                    regularization=0.1,
                                    iterations=15,
                                    calculate_training_loss=True,
                                    use_gpu=False)

# Обучение модели.
model_als.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

# Выполнение прогноза.
result = result_test.copy()
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model_als, N=5))
metric = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
del result

# Проверка: вывод метрики качества.
metric

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: total: 20.8 s
Wall time: 13.1 s


0.16454456415279137

### get_similar_items_recommendation

In [23]:
# Функция для подбора похожих продуктов.
def get_similar_items_recommendation(user, result_train, model, id_to_itemid, itemid_to_id, filter_items=[], N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    
    # На вход функции поступает список купленных продуктов, из него выбираются топ-N купленных продуктов.
    user_items = [*result_train.loc[result_train['user_id'] == 1, 'actual']][0][:N]
    
    # Список для хранения похожих продуктов.
    res = []
    
    # Для каждого продукта из топа:
    for item in user_items:
        # выбрать второй наиболее похожий продукт, так как первый - это сам рассматриваемый продукт.
        item_rec = id_to_itemid[model.similar_items(itemid_to_id[item], N=2)[1][0]]
        
        # Если продукт попал в категорию "прочие",
        if item_rec in filter_items:
            # выбрать следующий продукт.
            item_rec = id_to_itemid[model.similar_items(itemid_to_id[item], N=3)[2][0]]
        
        res.append(id_to_itemid[model.similar_items(itemid_to_id[item], N=2)[1][0]])
    
    return res

In [24]:
# Формирование проверочной таблицы на обучающей выборке.
result_train = df_train.copy().groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']
result_train.head(2)

,user_id,actual
0,1,"[999999, 840361, 845307, 852014, 856942, 91267..."
1,2,"[854852, 930118, 1077555, 1098066, 999999, 556..."


In [25]:
# Топ 5 товаров первого пользователя.
[*result_train.loc[result_train['user_id'] == 1, 'actual']][0][:5]

array([999999, 840361, 845307, 852014, 856942], dtype=int64)

In [26]:
# Подбор похожих товаров на основе купленных.
get_similar_items_recommendation(1, result_train, model_als, id_to_itemid, itemid_to_id, [999999], 5)

[1082185, 904360, 9677236, 924760, 934399]

In [27]:
%%time

# Выполнение прогноза.
result = result_test.copy()
result['sir'] = result['user_id'].apply(lambda x:
                                        get_similar_items_recommendation(x,
                                                                         result_train,
                                                                         model_als,
                                                                         id_to_itemid,
                                                                         itemid_to_id,
                                                                         [999999],
                                                                         5))

metric = result.apply(lambda row: precision_at_k(row['sir'], row['actual']), axis=1).mean()
del result

# Проверка: вывод метрики качества.
metric

CPU times: total: 3.91 s
Wall time: 3.2 s


0.07482859941234085

### get_similar_users_recommendation

In [28]:
%%time

# Инициализация модели для собственных прогнозов пользователя.
own_recommender = ItemItemRecommender(K=1)

# Обучение модели.
own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr(),
                    show_progress=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: total: 1.52 s
Wall time: 621 ms


In [29]:
# Проверка работы модели:
# прогноз двух продуктов для первого пользователя.
own_recommender.recommend(userid=1,
                          user_items=csr_matrix(user_item_matrix).tocsr(),
                          N=2,
                          filter_items=[itemid_to_id[999999]])

[(4346, 41882813736365.0), (4339, 383519267.0)]

In [30]:
# Функция для подбора продуктов похожих пользователей.
def get_similar_users_recommendation(user, model_als, model_own, user_item_matrix, filter_items=[], N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    # Формирование списка N похожих пользователей, кроме первого пользователя - это сам рассматриваемый пользователь.
    list_similar_users = [user_id for user_id, _ in model_als.similar_users(user, N + 1)[1:]]
    
    # Список для хранения товаров похожих пользователей.
    res = []
    
    # Для каждого похожего пользователя:
    for similar_user in list_similar_users:
        # выполнить прогноз N продуктов,
        recs = model_own.recommend(userid=similar_user,
                                   user_items=user_item_matrix,
                                   N=N,
                                   filter_items=filter_items)
        
        # выбрать первый продукт.
        item = recs[0][0]
        
        # Для всех предсказанных продуктов текущего похожего пользователя:
        for i in range(len(recs)):
            # если продукт уже в списке предсказанных:
            if item in res:
                # выбрать следующий продукт.
                item = recs[i][0]

        # добавить продукт в список для рекомендации.
        res.append(item)
    
    return res

In [31]:
%%time

# Проверка работы функции:
# прогноз продуктов пяти наиболее похожих пользователей для 1-го пользователя,
recs = get_similar_users_recommendation(user=1,
                                        model_als=model_als,
                                        model_own=own_recommender,
                                        user_item_matrix=csr_matrix(user_item_matrix).tocsr(),
                                        filter_items=[itemid_to_id[999999]])

# преобразование id продукта к itemid,
recs = [id_to_itemid[x] for x in recs]

# вывод продуктов пяти наиболее похожих пользователей для 1-го пользователя.
recs

CPU times: total: 234 ms
Wall time: 229 ms


[6534178, 6533889, 6534166, 6533889, 6533889]

Продукты могут повторяться, так как модель "own_recommender" не всегда может спрогнозировать достаточное количество уникальных продуктов.

In [32]:
%%time

# Выполнение прогноза.
# result = result_test.copy()
result = result_test.iloc[0:20, :].copy()
result['sur'] = result['user_id'].apply(lambda x:
                                        get_similar_users_recommendation(user=1,
                                                                         model_als=model_als,
                                                                         model_own=own_recommender,
                                                                         user_item_matrix=csr_matrix(user_item_matrix).tocsr(),
                                                                         filter_items=[itemid_to_id[999999]]))

metric = result.apply(lambda row: precision_at_k(row['sur'], row['actual']), axis=1).mean()
del result

# Проверка: вывод метрики качества.
metric

CPU times: total: 4.39 s
Wall time: 4.4 s


0.0

## 4. Проверьте, что все модули корректно импортируются.

In [33]:
!py -m build

* Creating venv isolated environment...
* Installing packages in isolated environment... (setuptools>=42)
* Getting dependencies for sdist...
running egg_info
writing src\package_4th_homework.egg-info\PKG-INFO
writing dependency_links to src\package_4th_homework.egg-info\dependency_links.txt
writing top-level names to src\package_4th_homework.egg-info\top_level.txt
reading manifest file 'src\package_4th_homework.egg-info\SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src\package_4th_homework.egg-info\SOURCES.txt'
* Building sdist...
running sdist
running egg_info
writing src\package_4th_homework.egg-info\PKG-INFO
writing dependency_links to src\package_4th_homework.egg-info\dependency_links.txt
writing top-level names to src\package_4th_homework.egg-info\top_level.txt
reading manifest file 'src\package_4th_homework.egg-info\SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src\package_4th_homework.egg-info\SOURCES.txt'
running check
creating package_4

In [34]:
# Установка пакета с разработанными функциями.
!pip install dist\package_4th_homework-1.tar.gz

Processing c:\users\zsv\pycharmprojects\geekbrains\recommender systems\4th lesson\dist\package_4th_homework-1.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for package-4th-homework: filename=package_4th_homework-1-py3-none-any.whl size=4457 sha256=1def2847033abe01262a1a44a6b91c2f53c02917d0f9da11fc3945bd5c777208
  Stored in directory: c:\users\zsv\appdata\local\pip\cache\wheels\d6\28\65\19efff1ad31bdc425fa53d2f808c6453d09f8d2abb599e945d
Successfully built package-4th-homework
  Attempting uninstall: package-4th-homework
    Found existing installation: package-4th-homework 1
    Un

In [35]:
from package_4th_homework import utils
from package_4th_homework import recommenders

In [36]:
%%time


df_train_2 = utils.prefilter_items(df_train)

print(f'Размерность таблицы до фильтрации:\t{df_train.shape}\n' \
      f'Размерность таблицы после фильтрации:\t{df_train_2.shape}\n' \
      f'\n' \
      f'Количество продуктов до фильтрации:\t{df_train["item_id"].nunique()}\n' \
      f'Количество продуктов после фильтрации:\t{df_train_2["item_id"].nunique()}\n'
     )

Размерность таблицы до фильтрации:	(2278490, 12)
Размерность таблицы после фильтрации:	(1313240, 12)

Количество продуктов до фильтрации:	5001
Количество продуктов после фильтрации:	4674

CPU times: total: 1.81 s
Wall time: 1.81 s


In [37]:
# Подбор похожих товаров на основе купленных.
recommenders.get_similar_items_recommendation(1, result_train, model_als, id_to_itemid, itemid_to_id, ['999999'], 5)

[1082185, 904360, 9677236, 924760, 934399]

In [38]:
%%time

# Проверка работы функции:
# прогноз продуктов пяти наиболее похожих пользователей для 1-го пользователя,
recs = recommenders.get_similar_users_recommendation(user=1,
                                                     model_als=model_als,
                                                     model_own=own_recommender,
                                                     user_item_matrix=csr_matrix(user_item_matrix).tocsr(),
                                                     filter_items=[itemid_to_id[999999]])

# преобразование id продукта к itemid,
recs = [id_to_itemid[x] for x in recs]

# вывод продуктов пяти наиболее похожих пользователей для 1-го пользователя.
recs

CPU times: total: 250 ms
Wall time: 246 ms


[6534178, 6533889, 6534166, 6533889, 6533889]